In [ ]:
# ============================================================================
# CELL 1: Install Required Libraries
# ============================================================================
# This installs all the packages we need for fine-tuning
# Takes about 1-2 minutes

!pip install -q transformers accelerate bitsandbytes peft datasets

In [ ]:
# ============================================================================
# CELL 2: Load Base Model and Tokenizer
# ============================================================================
# This loads TinyLlama (1.1B parameters) - a small but capable model
# We use 4-bit quantization to save memory (load_in_4bit=True)

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

BASE_MODEL = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

print("Loading tokenizer and model...")
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token  # Required for batching

model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    device_map="auto",
    load_in_4bit=True,
    torch_dtype=torch.float16
)

print("✓ Model loaded successfully!")
print(f"Model is on device: {model.device}")

In [ ]:
# ============================================================================
# CELL 3: Create Training Dataset
# ============================================================================
# 50 examples covering Salesforce Agentforce, NetSuite AI, and SaaS Revenue (ASC 606)

training_data = [
    # SALESFORCE AGENTFORCE (20 examples)
    {
        "instruction": "What is Salesforce Agentforce?",
        "output": "Agentforce is Salesforce's autonomous AI agent platform that allows businesses to build and deploy AI agents to handle tasks like customer service, sales, and operations without human intervention."
    },
    {
        "instruction": "What are the main types of Agentforce agents?",
        "output": "The main types are Service Agent for customer support, Sales Development Representative (SDR) Agent for lead qualification, Personal Shopper Agent for e-commerce, and Campaign Agent for marketing automation."
    },
    {
        "instruction": "How do Agentforce topics work?",
        "output": "Topics define what an agent knows about. They're configured with data sources, instructions, and guardrails. When a user asks a question, the agent uses RAG to retrieve relevant information from topics to generate responses."
    },
    {
        "instruction": "What is RAG in Agentforce?",
        "output": "RAG stands for Retrieval Augmented Generation. It allows agents to search through your company data in real-time and use that context to generate accurate, grounded responses rather than relying solely on the LLM's training."
    },
    {
        "instruction": "How do you trigger an Agentforce agent?",
        "output": "Agents can be triggered through multiple channels: Einstein Copilot, Flow automation, Apex code, Experience Cloud sites, or API calls. You configure triggers based on business logic like case creation or time-based events."
    },
    {
        "instruction": "What are Agentforce guardrails?",
        "output": "Guardrails are safety controls that prevent agents from taking unwanted actions. They include data access controls, action permissions, approval requirements, and content filters to ensure agents operate within defined boundaries."
    },
    {
        "instruction": "Can Agentforce agents escalate to humans?",
        "output": "Yes, agents can automatically escalate to human agents based on rules like customer sentiment, complexity thresholds, specific keywords, or when the agent's confidence score falls below a set level."
    },
    {
        "instruction": "How does Agentforce handle multiple languages?",
        "output": "Agentforce supports multilingual interactions through Einstein's language detection and translation capabilities. Agents can detect the user's language and respond accordingly, or you can configure language-specific topics."
    },
    {
        "instruction": "What data sources can Agentforce access?",
        "output": "Agents can access Salesforce objects, Knowledge articles, external data via MuleSoft or APIs, Data Cloud, document repositories, and custom data sources configured through Flow or Apex."
    },
    {
        "instruction": "How do you measure Agentforce performance?",
        "output": "Use Agent Analytics to track metrics like resolution rate, average handling time, CSAT scores, escalation rate, accuracy of responses, and conversation volume. These are available in the Einstein Trust Layer dashboard."
    },
    {
        "instruction": "What is the Einstein Trust Layer?",
        "output": "The Einstein Trust Layer is Salesforce's security architecture for AI. It includes data masking, zero retention policies with LLM providers, toxicity detection, audit logs, and ensures your data isn't used to train external models."
    },
    {
        "instruction": "Can you customize Agentforce prompts?",
        "output": "Yes, you can customize system prompts, topic instructions, and response templates through Prompt Builder. This allows you to control the agent's tone, format, and specific behaviors for your use case."
    },
    {
        "instruction": "How does Agentforce integrate with Flow?",
        "output": "Agents can invoke Flow actions to perform complex operations like updating records, sending emails, or calling external systems. Flows can also trigger agents based on business logic or schedule."
    },
    {
        "instruction": "What's the difference between Einstein Copilot and Agentforce?",
        "output": "Einstein Copilot is an AI assistant that helps users within Salesforce with recommendations and automation. Agentforce agents are autonomous and can handle end-to-end processes independently, including customer-facing interactions."
    },
    {
        "instruction": "How do you handle agent authentication?",
        "output": "Agents inherit permissions from their configured service account or use guest user profiles for public-facing agents. You control data access through profiles, permission sets, and sharing rules like any Salesforce user."
    },
    {
        "instruction": "Can Agentforce agents learn from interactions?",
        "output": "Agents improve through feedback loops where human reviews, CSAT ratings, and correction data are fed back into the system. This helps refine topics, prompts, and retrieval accuracy over time."
    },
    {
        "instruction": "What's required to deploy an Agentforce agent?",
        "output": "You need Einstein 1 license, configured topics with data sources, defined triggers and channels, set guardrails and permissions, tested responses, and established monitoring and escalation paths."
    },
    {
        "instruction": "How does Agentforce handle unstructured data?",
        "output": "Agents use semantic search and vector embeddings to understand unstructured content like PDFs, emails, and chat transcripts. The RAG system converts queries and documents into embeddings to find relevant context."
    },
    {
        "instruction": "What are Agentforce action groups?",
        "output": "Action groups are collections of related actions an agent can perform, like 'Case Management' actions including create case, update status, and send notification. They help organize and control agent capabilities."
    },
    {
        "instruction": "Can you A/B test Agentforce configurations?",
        "output": "Yes, you can create multiple versions of topics or prompts and route traffic between them. Use Agent Analytics to compare performance metrics and determine which configuration performs better."
    },

    # NETSUITE AI & AUTOMATION (15 examples)
    {
        "instruction": "What AI features does NetSuite offer in 2025?",
        "output": "NetSuite offers Text Enhancer for AI-generated item descriptions, Intelligent Recommendations for next-best actions, Analytics Warehouse with ML predictions, Document Processing automation, and Smart Alerts for anomaly detection."
    },
    {
        "instruction": "What is NetSuite Text Enhancer?",
        "output": "Text Enhancer uses generative AI to automatically create product descriptions, marketing copy, and item details from minimal input. It helps maintain consistency and saves time on catalog management."
    },
    {
        "instruction": "How does NetSuite Analytics Warehouse use ML?",
        "output": "Analytics Warehouse applies machine learning to historical data to generate demand forecasts, identify trends, predict cash flow, and surface anomalies. It provides predictive insights within dashboards and reports."
    },
    {
        "instruction": "What is NetSuite Intelligent Recommendations?",
        "output": "It's an AI system that suggests next-best actions to users based on context, like recommending approval workflows, suggesting reorder points, or highlighting exceptions that need attention."
    },
    {
        "instruction": "How does NetSuite automate document processing?",
        "output": "NetSuite uses OCR and AI to extract data from invoices, receipts, and purchase orders automatically. It can match documents to transactions, validate data, and route for approval without manual entry."
    },
    {
        "instruction": "Can NetSuite predict inventory needs?",
        "output": "Yes, through Analytics Warehouse and demand planning features. It analyzes historical sales, seasonality, and trends to forecast future demand and recommend optimal stock levels and reorder points."
    },
    {
        "instruction": "What are NetSuite Smart Alerts?",
        "output": "Smart Alerts use anomaly detection to notify users of unusual patterns like unexpected expense spikes, revenue drops, or inventory discrepancies. They help catch issues before they become problems."
    },
    {
        "instruction": "How does NetSuite handle multi-subsidiary automation?",
        "output": "NetSuite's OneWorld feature automates intercompany transactions, currency conversions, and consolidation across subsidiaries. AI recommendations help optimize transfer pricing and identify consolidation exceptions."
    },
    {
        "instruction": "What is SuiteAnalytics Connect?",
        "output": "It's a JDBC/ODBC driver that allows external BI tools and AI platforms to query NetSuite data directly. This enables integration with advanced analytics and ML tools outside NetSuite."
    },
    {
        "instruction": "Can NetSuite automate revenue recognition?",
        "output": "Yes, NetSuite's Advanced Revenue Management module automates ASC 606 compliance by creating revenue schedules, handling performance obligations, and generating recognition entries based on configured rules."
    },
    {
        "instruction": "How does NetSuite AI improve quote-to-cash?",
        "output": "AI features streamline CPQ with smart product recommendations, automated pricing, contract generation using templates, and intelligent approval routing. Analytics predict deal closure probability and identify bottlenecks."
    },
    {
        "instruction": "What machine learning models does NetSuite use?",
        "output": "NetSuite primarily uses time-series forecasting models, classification algorithms for document processing, clustering for customer segmentation, and anomaly detection models for alerts."
    },
    {
        "instruction": "Can you train custom AI models in NetSuite?",
        "output": "NetSuite doesn't support custom model training natively. However, you can integrate external ML platforms via SuiteCloud Platform, REST APIs, or SuiteAnalytics Connect to apply custom models to NetSuite data."
    },
    {
        "instruction": "How does NetSuite ensure AI data security?",
        "output": "NetSuite applies role-based access controls to AI features, encrypts data in transit and at rest, maintains SOC 2 compliance, and ensures AI processing doesn't expose data beyond user permissions."
    },
    {
        "instruction": "What's new in NetSuite's 2025 AI roadmap?",
        "output": "NetSuite is expanding generative AI for report creation, natural language querying of data, enhanced forecasting models, and deeper integration with analytics tools. Focus is on making ERP more intuitive and predictive."
    },

    # SAAS REVENUE - ASC 606 (15 examples)
    {
        "instruction": "What is ASC 606?",
        "output": "ASC 606 is the revenue recognition standard that requires companies to recognize revenue when control of goods or services transfers to the customer, not simply when cash is received. It replaced ASC 605 in 2018."
    },
    {
        "instruction": "What are the 5 steps of ASC 606?",
        "output": "The five steps are: 1) Identify the contract with a customer, 2) Identify performance obligations, 3) Determine the transaction price, 4) Allocate price to performance obligations, 5) Recognize revenue when obligations are satisfied."
    },
    {
        "instruction": "What is a performance obligation in ASC 606?",
        "output": "A performance obligation is a promise to transfer a distinct good or service to a customer. In SaaS, common obligations include software access, implementation services, training, and ongoing support."
    },
    {
        "instruction": "How do you recognize SaaS subscription revenue under ASC 606?",
        "output": "SaaS subscriptions are typically recognized ratably over the subscription period because the performance obligation (providing software access) is satisfied over time as the customer consumes the service."
    },
    {
        "instruction": "What is deferred revenue?",
        "output": "Deferred revenue is cash received from customers for services not yet delivered. It's recorded as a liability on the balance sheet and recognized as revenue when the performance obligation is satisfied."
    },
    {
        "instruction": "How do you handle upfront implementation fees under ASC 606?",
        "output": "If implementation creates a distinct service, recognize revenue when complete. If it's not distinct from the SaaS subscription, defer the fee and recognize it over the subscription term along with subscription revenue."
    },
    {
        "instruction": "What is the difference between ARR and revenue?",
        "output": "ARR (Annual Recurring Revenue) is a forward-looking metric showing annualized contract value. Revenue is what's actually recognized per GAAP based on performance obligations satisfied. ARR helps predict future revenue."
    },
    {
        "instruction": "How do you allocate transaction price in multi-element arrangements?",
        "output": "Allocate the total contract price to each performance obligation based on standalone selling prices (SSP). If SSP isn't observable, estimate it using cost-plus, market assessment, or residual approaches."
    },
    {
        "instruction": "What triggers a contract modification under ASC 606?",
        "output": "A modification occurs when contract scope or price changes and both parties approve. Treat it as a separate contract if it adds distinct goods/services at SSP. Otherwise, adjust the existing contract prospectively or retrospectively."
    },
    {
        "instruction": "How do you handle variable consideration in SaaS?",
        "output": "Include variable consideration (like usage fees or performance bonuses) in transaction price only to the extent it's probable that a significant revenue reversal won't occur. Update estimates each period."
    },
    {
        "instruction": "What is MRR in SaaS businesses?",
        "output": "MRR (Monthly Recurring Revenue) is the normalized monthly value of all active subscriptions. It's calculated by dividing annual contracts by 12 and summing all monthly subscriptions. MRR helps track business momentum."
    },
    {
        "instruction": "How do you recognize revenue for usage-based pricing?",
        "output": "Recognize usage-based revenue as the customer uses the service, since that's when the performance obligation is satisfied. This requires tracking actual usage and applying the contracted rates each period."
    },
    {
        "instruction": "What are contract assets vs receivables under ASC 606?",
        "output": "A receivable is an unconditional right to payment. A contract asset is revenue recognized before you have an unconditional right to payment (like when performance precedes billing milestones)."
    },
    {
        "instruction": "How do refund provisions affect revenue recognition?",
        "output": "Refund provisions create variable consideration. Estimate expected refunds using historical data and reduce revenue recognized accordingly. Update the estimate each period and adjust revenue if expectations change."
    },
    {
        "instruction": "What is the practical expedient for contract costs?",
        "output": "ASC 606 allows expensing contract acquisition costs immediately if the amortization period would be one year or less. This simplifies accounting for monthly or short-term SaaS contracts."
    }
]

print(f"✓ Created training dataset with {len(training_data)} examples")
print(f"  - Salesforce Agentforce: 20 examples")
print(f"  - NetSuite AI: 15 examples")
print(f"  - SaaS Revenue (ASC 606): 15 examples\n")

In [ ]:
# ============================================================================
# CELL 4: Prepare Model and Train with LoRA
# ============================================================================
# LoRA (Low-Rank Adaptation) is an efficient fine-tuning method
# Instead of updating all 1.1B parameters, we add small adapter layers
# This is much faster and uses less memory

from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import Trainer, TrainingArguments
import torch
import os

# Disable wandb to avoid API key prompts
os.environ["WANDB_DISABLED"] = "true"

# Format data as conversational prompts
def format_instruction(example):
    return f"User: {example['instruction']}\nAssistant: {example['output']}"

formatted_texts = [format_instruction(ex) for ex in training_data]

# Tokenize all examples
print("Tokenizing training data...")
encodings = tokenizer(
    formatted_texts,
    padding="max_length",
    truncation=True,
    max_length=256,  # Shorter for faster training
    return_tensors="pt"
)

# Create PyTorch dataset with labels
class InstructionDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.input_ids = encodings["input_ids"]
        self.attention_mask = encodings["attention_mask"]

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            "input_ids": self.input_ids[idx],
            "attention_mask": self.attention_mask[idx],
            "labels": self.input_ids[idx].clone()  # For causal LM, labels = input_ids
        }

train_dataset = InstructionDataset(encodings)
print(f"✓ Dataset ready with {len(train_dataset)} examples\n")

# Prepare model for LoRA training
print("Preparing model for LoRA fine-tuning...")
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

# LoRA configuration - these are the adapter parameters
lora_config = LoraConfig(
    r=8,  # Rank of the adapter matrices
    lora_alpha=16,  # Scaling factor
    lora_dropout=0.05,  # Dropout for regularization
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "v_proj"]  # Which layers to add adapters to
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()  # Shows we're only training ~0.1% of parameters

# Training configuration
training_args = TrainingArguments(
    output_dir="./training_output",
    num_train_epochs=3,  # Number of times to go through the dataset
    per_device_train_batch_size=1,  # Smaller batch size for stability
    gradient_accumulation_steps=4,  # Effective batch size = 1 * 4 = 4
    learning_rate=2e-4,
    fp16=True,  # Use mixed precision for speed
    logging_steps=5,
    save_strategy="no",  # Don't save checkpoints during training
    optim="paged_adamw_8bit",  # Memory-efficient optimizer
    report_to="none"  # Disable reporting to wandb/tensorboard
)

# Create trainer and start training
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset
)

print("\n" + "="*60)
print("STARTING TRAINING - This will take about 5-10 minutes")
print("="*60 + "\n")

trainer.train()

print("\n" + "="*60)
print("✓ TRAINING COMPLETE!")
print("="*60 + "\n")

In [ ]:
# ============================================================================
# CELL 5: Test the Fine-Tuned Model and Save Everything
# ============================================================================

print("Testing the fine-tuned model...\n")
print("="*60)

# Put model in evaluation mode
model.eval()

# Test with questions from each domain
test_questions = [
    "User: What is RAG in Agentforce?\nAssistant:",
    "User: What is NetSuite Text Enhancer?\nAssistant:",
    "User: What are the 5 steps of ASC 606?\nAssistant:",
    "User: How do you recognize SaaS subscription revenue?\nAssistant:"
]

for question in test_questions:
    print(f"\n📝 Question: {question.split('Assistant:')[0].replace('User: ', '').strip()}")

    inputs = tokenizer(question, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=100,
            temperature=0.7,
            do_sample=True,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Extract just the assistant's response
    assistant_response = response.split("Assistant:")[-1].strip()
    print(f"🤖 Answer: {assistant_response}")
    print("-" * 60)

print("\n" + "="*60)
print("SAVING MODEL")
print("="*60 + "\n")

# Save the LoRA adapter and tokenizer
SAVE_DIR = "tinyllama-revops-finetuned"

model.save_pretrained(SAVE_DIR)
tokenizer.save_pretrained(SAVE_DIR)

print(f"✓ Model saved to './{SAVE_DIR}/'")
print("\nWhat was saved:")
print("  - adapter_config.json (LoRA configuration)")
print("  - adapter_model.safetensors (trained weights)")
print("  - tokenizer files")
print("\n" + "="*60)
print("✅ ALL DONE!")
print("="*60)
print("\nNext steps:")
print("1. Download the folder from Colab (Files tab on left)")
print("2. Create a GitHub repo and upload these files")
print("3. Add a README.md explaining your project")
print("4. Post on LinkedIn with examples of the model's responses!")
print("\nThe base model is TinyLlama-1.1B-Chat-v1.0")
print("Your fine-tuned adapter is ready to share! 🚀")

In [ ]:
!zip -r tinyllama-revops-finetuned.zip tinyllama-revops-finetuned

In [ ]:
# CELL 6 — Upload to Hugging Face (safe; upload-only)
!pip -q install huggingface_hub
from huggingface_hub import login, create_repo, upload_folder

login()  # paste token in the popup
REPO_ID = "Builder123/tinyllama-revops-finetuned"   # change if you prefer

create_repo(REPO_ID, repo_type="model", exist_ok=True)
upload_folder(repo_id=REPO_ID, folder_path="tinyllama-revops-finetuned")

print("Done → https://huggingface.co/" + REPO_ID)
